In [ ]:
from board import Board
from quantity import Quantity
from player import Player

In [ ]:
def learning(board, quantity, player1, player2, stories=2000):
    for story in range(stories):
        board.reset()
        quantity.reset()
        while True:
            p1_ret = player1.put(board, quantity, train=True)
            p2_ret = player2.put(board, quantity, train=True)
            if p1_ret == 0 and p2_ret == 0:
                break

In [ ]:
# 学習
b = Board(6)
q = Quantity(6)
p1 = Player(Board.BLACK)
p2 = Player(Board.WHITE)
learning(b, q, p1, p2)

In [ ]:
# 個別対戦用にリセット
b.reset()
q.reset()
print(b.board)